In [10]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'plant-disease-recognition-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1447507%2F2394131%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T182737Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D98c8bebf11a308e6a8d8bed10287976c3208cc60eaa20385d56e9e43284f0a5d9a05d11d3a94acfcc625b0f094dc1317a02f5cb800beea114edc30442c8b1c047637ac21c1c65d553a6479980c31ca370c6f411cb3d2ff852e2ca97e2eb3386ab2dc83308d7b71fc52bd43c7777e76b28708fd824e44e5701b87d3220b8a08f79e66bea0027d5f8cbdb8f96ffe52c448a6315ea870015bab3345adbceba85b24fc4ca891ef6287eb3ed8f7ccbfc97f357fc48f624bc1374c1b33f60844ab5ae9769df0032f0e9db870408bbe69248b28e2b6bd14382444b1fd97165ccdb0b52640d26bb3f2ea7cdae59a085e0b774a23b7093dcfe024c80577316bcf5ad5085f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1339624018 bytes downloaded
Downloaded and uncompressed: plant-disease-recognition-dataset
Data source import complete.


In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/8219aa662e8fd59d.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/8030a3a79fca6abb.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/8152cfbd5a28b5d2.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/881b4dba7731b362.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/8196e2d8b66ed721.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/885f743de596e442.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/955f62896264dbb2.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/902b269c0ddbe3e5.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/929b16251fcaafc4.jpg
/kaggle/input/plant-disease-recognition-dataset/Validation/Validation/Rust/851edcf908ed2e1c.jpg
/kaggle/input/plant-disease-recognition-

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [13]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [14]:
train_dataset = train.flow_from_directory('/kaggle/input/plant-disease-recognition-dataset/Train/Train',
#                                           target_size=(2672, 4000),
                                          target_size=(256,256),
                                          batch_size=5,
                                          class_mode="binary"
                                         )

Found 1322 images belonging to 3 classes.


In [15]:
validation_dataset = train.flow_from_directory('/kaggle/input/plant-disease-recognition-dataset/Validation/Validation',
                                          target_size=(256,256),
                                          batch_size=5,
                                          class_mode="binary"
                                         )

Found 60 images belonging to 3 classes.


In [16]:
model = models.Sequential()

In [17]:
model.add(layers.Conv2D(32,(3,3), activation = 'relu',input_shape = (256, 256, 3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = 'relu' ))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = 'relu' ))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))


In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# Train the model
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)

Epoch 1/10
265/265 [==============================] - 79s 278ms/step - loss: 0.7475 - accuracy: 0.6558 - val_loss: 0.4014 - val_accuracy: 0.8500
Epoch 2/10
265/265 [==============================] - 75s 281ms/step - loss: 0.4688 - accuracy: 0.8495 - val_loss: 0.8759 - val_accuracy: 0.7667
Epoch 3/10
265/265 [==============================] - 75s 282ms/step - loss: 0.2206 - accuracy: 0.9395 - val_loss: 0.2993 - val_accuracy: 0.9000
Epoch 4/10
265/265 [==============================] - 75s 283ms/step - loss: 0.2039 - accuracy: 0.9342 - val_loss: 0.2985 - val_accuracy: 0.8667
Epoch 5/10
265/265 [==============================] - 81s 304ms/step - loss: 0.1865 - accuracy: 0.9410 - val_loss: 0.7311 - val_accuracy: 0.8333
Epoch 6/10
265/265 [==============================] - 84s 316ms/step - loss: 0.1400 - accuracy: 0.9539 - val_loss: 0.4445 - val_accuracy: 0.8833
Epoch 7/10
265/265 [==============================] - 75s 284ms/step - loss: 0.1258 - accuracy: 0.9660 - val_loss: 0.6979 - val_ac

In [25]:
# model.save("/model.keras")

In [22]:
test = ImageDataGenerator(rescale=1/255)

In [23]:
train_dataset = train.flow_from_directory('/kaggle/input/plant-disease-recognition-dataset/Test/Test',
#                                           target_size=(2672, 4000),
                                          target_size=(256,256),
                                          batch_size=5,
                                          class_mode="binary"
                                         )

Found 150 images belonging to 3 classes.


In [26]:
model.evaluate(train_dataset)


30/30 [==============================] - 8s 269ms/step - loss: 0.4484 - accuracy: 0.9133


[0.448444664478302, 0.9133333563804626]